In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer #to find out the base root of the words 
from nltk.corpus import stopwords
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer #find TF*IDF  (Term Frequency(on one by one sentence ) = rep/total word , Inverse Document Frequency(on whole dataset) = log(total sen/rep))
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score#cration histogram ,count frequency ,make all words as feature
# from nltk.metrics.distance import jaccard_distance
# from nltk.util import ngrams
# from nltk.metrics.distance  import edit_distance
import warnings
warnings.filterwarnings("ignore")

In [2]:
nltk.download('punkt')  
nltk.download('stopwords')
nltk.download('words')



[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /usr/share/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
df = pd.read_csv("../input/yahoo-troll-question-detection/train_df.csv")
dt = pd.read_csv("../input/yahoo-troll-question-detection/test_df.csv")



In [4]:
whole_input_data_after_preprocessing=[]

In [5]:
def Preprocessing_of_data(start,end):
#     ps=PorterStemmer()
    for i in range(start,end):  
        words=re.sub('[^a-zA-Z]',' ',df['question_text'][i]) #removing of all other char other then alphabet a to z and A to Z 
        words=words.lower() #convert all words to the lowercase
#         words=words.split()
#         words=getCorrectWords(words)
#         words=[ps.stem(x) for x in words if x  not in set(stopwords.words('english')) ]
#         words=' '.join(words)
        whole_input_data_after_preprocessing.append(words)

In [6]:
Preprocessing_of_data(0,1000000)

In [7]:
len(whole_input_data_after_preprocessing)

1000000

In [8]:
main_test_input=[]
# ps=PorterStemmer()
for i in range(0,len(dt)):  
    m_words=re.sub('[^a-zA-Z]',' ',dt['question_text'][i]) #removing of all other char other then alphabet a to z and A to Z 
    m_words=m_words.lower() #convert all words to the lowercase
#     m_words=m_words.split()
#     m_words=[ps.stem(x) for x in m_words if x  not in set(stopwords.words('english')) ]
#     m_words=' '.join(m_words)
    main_test_input.append(m_words)
    

In [9]:
len(main_test_input)

306122

In [10]:
tr_i=whole_input_data_after_preprocessing
te_i=main_test_input
combination_full_test=tr_i+te_i
y=df['target']
y=y[:len(tr_i)]

In [11]:

main_250000_cv=CountVectorizer()
till_full_TFIDF_X=main_250000_cv.fit_transform(combination_full_test)

In [12]:
print(till_full_TFIDF_X.shape)
print(len(y))


(1306122, 178847)
1000000


In [13]:
main_full_X_train=till_full_TFIDF_X[:1000000]
main_full_X_test=till_full_TFIDF_X[1000000:]

In [14]:
print(main_full_X_train.shape)
print(main_full_X_test.shape)

(1000000, 178847)
(306122, 178847)


In [15]:
from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(max_iter=200)
# main_full_yahoo_model=lr.fit(main_full_X_train,y)

In [16]:
logModel = LogisticRegression()

In [17]:
w={0:0.2,1:0.8}

In [18]:
lr = LogisticRegression(max_iter=1000,n_jobs=-1,class_weight =w,solver='saga')
main_full_yahoo_model=lr.fit(main_full_X_train, y)

In [19]:
main_full_y_pred=main_full_yahoo_model.predict(main_full_X_test)

In [20]:
output_full_datfrme=pd.DataFrame({ 'qid': dt['qid'] ,'target': np.array(main_full_y_pred)})

In [21]:
output_full_datfrme.to_csv('count_lg_whole_15_85_saga.csv',index=False)